# 코루틴

## 코루틴 사용하기

 - 다음과 같이 calc 함수 안에서 add 함수를 호출했을 때 add 함수가 끝나면 다시 calc 함수로 돌아옵니다.
 - 특히 add 함수가 끝나면 이 함수에 들어있던 변수와 계산식은 모두 사라집니다.

In [1]:
def add(a, b):
    c = a + b    # add 함수가 끝나면 변수와 계산식은 사라짐
    print(c)
    print('add 함수')
 
def calc():
    add(1, 2)    # add 함수가 끝나면 다시 calc 함수로 돌아옴
    print('calc 함수')
 
calc()

3
add 함수
calc 함수


![](./image/coroutine_1.jpg)
- 이 소스코드에서 calc가 메인 루틴(main routine)이면 add는 calc의 서브 루틴(sub routine)입니다. 
- 메인 루틴에서 서브 루틴을 호출하면 서브 루틴의 코드를 실행 한 뒤 다시 메인 루틴으로 돌아옵니다.
  - 서브 루틴이 끝나면 서브 루틴의 내용은 모두 사라집니다. 즉, 서브 루틴은 메인 루틴에 종속된 관계입니다.

<br>

- 코루틴( coroutine)은 cooperative routine을 의미하는데 서로 협력하는 루틴이라는 뜻입니다.
- 즉 메인 루틴과 서브 루틴처럼 종속된 관계가 아니라 서로 대등한 관계이며 특정 시점에 상대방의 코드를 실행합니다.

![](./image/coroutine_2.jpg)

- 코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행합니다. 
- 따라서 코루틴이 종료되지 않았으므로 코루틴의 내용도 계속 유지됩니다.
- 일반 함수를 호출하면 코드를 한 번만 실행할 수 있지만, 코루틴은 코드를 여러 번 실행할 수 있습니다. 
- 참고로 함수의 코드를 실행하는 지점을 진입점(entry point)이라고 하는데, 코루틴은 진입점이 여러개인 함수입니다.

### 코루틴에 값 보내기

- 코루틴은 제너레이터의 특별한 형태입니다. 제너레이터는 yield로 값을 발생시켰지만 코루틴은 yield로 값을 받아올 수 있습니다. 
- 다음과 같이 코루틴에 값을 보내면서 코드를 실행할 때는 send 메서드를 사용합니다. 
- 그리고 send 메서드가 보낸 값을 받아오려면 (yield) 형식으로 yield를 괄호로 묶어준 뒤 변수에 저장합니다.
  - 코루틴객체.send(값)
  - 변수 = (yield)

In [26]:
def number_coroutine():
    while True:        # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield)    # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)
 
co = number_coroutine()
next(co)      # 코루틴 안의 yield까지 코드 실행(최초 실행)
 
co.send(1)    # 코루틴에 숫자 1을 보냄
co.send(2)    # 코루틴에 숫자 2을 보냄
co.send(3)    # 코루틴에 숫자 3을 보냄

1
2
3


### yield from 활용

In [27]:
def sum(max):
    tot = 0
    for i in range(max):
       tot += i
       yield tot
    return tot


def coroutine():
    x = yield from sum(10)
    print('Total: {}'.format(x))

x = coroutine()
next(x)

0

In [31]:
next(x)

10

# asyncio

- asyncio(Asynchronous I/O)는 비동기 프로그래밍을 위한 모듈이며 CPU 작업과 I/O를 병렬로 처리하게 해줍니다.
  - 동기(synchronous) 처리는 특정 작업이 끝나면 다음 작업을 처리하는 순차처리 방식
  - 비동기(asynchronous) 처리는 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식
- 제너레이터 기반의 코루틴과 구분하기 위해 async def로 만든 코루틴은 네이티브 코루틴이라고 합니다
- asyncio를 사용하려면 다음과 같이 async def로 네이티브 코루틴을 만듭니다

In [32]:
# 코루틴_4.py

import asyncio
 
async def hello():    # async def로 네이티브 코루틴을 만듦
    print('Hello, world!')
 
loop = asyncio.get_event_loop()     # 이벤트 루프를 얻음
loop.run_until_complete(hello())    # run_until_complete에 코루틴 객체를 넣음. 네이티브 코루틴을 호출하면 코루틴 객체가 생성됨. hello가 끝날 때까지 기다림
loop.close()                        # 이벤트 루프를 닫음

RuntimeError: This event loop is already running

## await로 네이티브 코루틴 실행
- await 뒤에 코루틴 객체, 퓨처 객체, 태스크 객체를 지정하면 해당 객체가 끝날 때까지 기다린 뒤 결과를 반환합니다.
- await 키워드는 파이썬 3.5 이상부터 사용 가능, 3.4에서는 yield from을 사용

In [22]:
#코루틴_5.py
import asyncio
 
async def add(a, b):
    print('add: {0} + {1}'.format(a, b))
    await asyncio.sleep(1.0)    # 1초 대기. asyncio.sleep도 네이티브 코루틴
    return a + b    # 두 수를 더한 결과 반환
 
async def print_add(a, b):
    result = await add(a, b)    # await로 다른 네이티브 코루틴 실행하고 반환값을 변수에 저장
    print('print_add: {0} + {1} = {2}'.format(a, b, result))
 
loop = asyncio.get_event_loop()             # 이벤트 루프를 얻음
loop.run_until_complete(print_add(1, 2))    # print_add가 끝날 때까지 이벤트 루프를 실행
loop.close()                                # 이벤트 루프를 닫음                

RuntimeError: This event loop is already running

- add: 1 + 2가 출력되고 1초 뒤에 print_add: 1 + 2 = 3이 출력됩니다.
- print_add에서는 await로 add를 실행하고 반환값을 변수에 저장했습니다. 이렇게 코루틴 안에서 다른 코루틴을 실행할 때는 await를 사용합니다.
- add에서는 await asyncio.sleep(1.0)로 1초 대기한 뒤 return a + b로 두 수를 더한 결과를 반환합니다.

- 퓨처(asyncio.Future)는 미래에 할 일을 표현하는 클래스인데 할 일을 취소하거나 상태 확인, 완료 및 결과 설정에 사용합니다.
- 태스크(asyncio.Task)는 asyncio.Future의 파생 클래스이며 asyncio.Future의 기능과 실행할 코루틴의 객체를 포함하고 있습니다. 태스크는 코루틴의 실행을 취소하거나 상태 확인, 완료 및 결과 설정에 사용합니다.

## 비동기 확인



In [ ]:
# 코루틴_6.py
import asyncio
import datetime
import random
 
 
async def display_date(num, loop, ):
    end_time = loop.time() + 20.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(random.randint(0, 5))
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()



- 주어진 시간(초) 후에 완료되는 코루틴 (coroutine)입니다. 
- 식별자 수(num)와 이벤트 루프(loop)를 매개변수로 받아 현재 시간을 계속 출력하는 coroutine 인 display_date (num, loop) 을 만듭니다. 
- 코루틴이기 때문에 외부로 부터 값을 받아드리는 성질이 있습니다.
- 다음 asyncio.sleep() 함수 호출의 결과를 기다리기 위해 yield from(await) 를 사용했습니다. 
- 그래서 우리는 그것에 임의의 초(random.randint(0, 5))를 보내고 asyncio.ensure_future()를 사용하여 기본 이벤트 루프에서 코루틴의 실행을 스케쥴합니다. 
- 그런 다음 루프가 계속 실행되도록 요청 합니다. loop.run_forever()
- 출력을 보면 두 개의 coroutine이 동시에 실행됩니다.
- yield from 를 사용할 때, 이벤트 루프는 코루틴의 실행을 일시 중지하고 다른 루틴을 실행합니다. 
- 따라서 두 개의 코루틴이 동시에 실행됩니다 (그러나 잊이벤트 루프가 단일 스레드이기 때문에 병렬로 실행되지 않습니다).


## asyncio 사용한 웹페이지 크롤링

In [33]:
# 동기

from time import time
from urllib.request import Request, urlopen
 
urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]
 
begin = time()
result = []
for url in urls:
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})    # UA가 없으면 403 에러 발생
    response = urlopen(request)
    page = response.read()
    result.append(len(page))
 
print(result)
end = time()
print('실행 시간: {0:.3f}초'.format(end - begin))

[249346, 144238, 91260, 70772, 303172, 86680]
실행 시간: 8.430초


In [34]:
# 코루틴_7.py
## 비동기
from time import time
from urllib.request import Request, urlopen
import asyncio
 
urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]
 
async def fetch(url):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})    
    response = await loop.run_in_executor(None, urlopen, request)    # run_in_executor 사용
    page = await loop.run_in_executor(None, response.read)           
    return len(page)
 
async def main():
    futures = [asyncio.ensure_future(fetch(url)) for url in urls] # 태스크(퓨처) 객체를 리스트로 만듦
    result = await asyncio.gather(*futures)                # 결과를 한꺼번에 가져옴
    print(result)
 
begin = time()
loop = asyncio.get_event_loop()          # 이벤트 루프를 얻음
loop.run_until_complete(main())          # main이 끝날 때까지 기다림
loop.close()                             # 이벤트 루프를 닫음
end = time()
print('실행 시간: {0:.3f}초'.format(end - begin))

C:\Users\jk.lee\anaconda3\lib\tokenize.py:450: RuntimeWarning: coroutine 'main' was never awaited
  line = readline()


RuntimeError: This event loop is already running

- urlopen이나 response.read 같은 함수(메서드)는 결과가 나올 때까지 코드 실행이 중단(block)되는데 이런 함수들을 블로킹 I/O(blocking I/O) 함수라고 부릅니다. 
- 특히 네이티브 코루틴 안에서 블로킹 I/O 함수를 실행하려면 이벤트 루프의 run_in_executor 함수를 사용하여 다른 스레드에서 병렬로 실행시켜야 합니다.
- run_in_executor의 첫 번째 인수는 executor인데 함수를 실행시켜줄 스레드 풀 또는 프로세스 풀입니다.
- 여기서는 None을 넣어서 기본 스레드 풀을 사용합니다. 
- 그리고 두 번째 인수에는 실행할 함수를 넣고 
- 세 번째 인수부터는 실행할 함수에 들어갈 인수를 차례대로 넣어줍니다.
  - 이벤트루프.run_in_executor(None, 함수, 인수1, 인수2, 인수3)